<a href="https://colab.research.google.com/github/Vinamit/api_flask/blob/main/STS_Project_Groupe1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic textual similarity

This google collab presents the work of the following people
- **Marc Edouard**
- **Frédéric Lauron**
- **Yannick Letort**
- **Pierre de La Ville Montbazon**

The following code can be used to load the data or processed results in various parts of the project. Note that the folders DataPart1, DataPart2 and DataPart3 need to be in the same folder as this jupyter notebook.
### Part 1 data
```python
# Features
np.load("DataPart1/train_features.npy",allow_pickle=True).item()
np.load("DataPart1/dev_features.npy",allow_pickle=True).item()
np.load("DataPart1/test_features.npy",allow_pickle=True).item()

# Results
np.load("DataPart1/fullresults_part1.npy",allow_pickle=True).item()
```
### Part 2 data
For this part, the data is only in text format so you can load it from any text editor.

# Part 3 data
The DataPart3 folder contains the three datasets used to build our models as well as txt files for the sub-part 1, and .npy files for sub-part 3.

```python
# Load the translated phrases using google translate from deep-translate
np.load("DataPart3/translations_train.npy",allow_pickle=True).item()
np.load("DataPart3/translations_test.npy",allow_pickle=True).item()

# Load the preprocessed English dataset
np.load("DataPart3/translated_dataset_preprocessed_default.npy",allow_pickle=True).item()
np.load("DataPart3/translated_dataset_preprocessed_wordnet.npy",allow_pickle=True).item()

# Load the features
np.load("DataPart3/features_train_es.npy",allow_pickle=True).item()
np.load("DataPart3/features_test_es.npy",allow_pickle=True).item()
```







In [ ]:
! pip install regex
! pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.8 MB/s eta 0:00:00


In [ ]:

import os
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from string import punctuation
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine, euclidean
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression,RidgeCV,LassoCV,ElasticNetCV
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import jaccard_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestRegressor
from statistics import harmonic_mean
from nltk.translate.bleu_score import sentence_bleu
import math
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
from nltk.tag import pos_tag
import re
import unidecode
import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
import seaborn as sns
from itertools import combinations
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import euclidean, cityblock
from scipy.stats import entropy

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
! wget http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
! tar -xvzf Stsbenchmark.tar.gz

--2023-12-03 15:53:52--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2023-12-03 15:53:52--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K   437KB/s    in 0.9s    

2023-12-03 15:53:54 (437 KB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]

stsbenchmark/
stsbenchmark/readme.txt
stsbenchmark/sts-test.csv
stsbenchmark/correlation.pl
stsbenchmark/LICENSE.txt
stsb

In [ ]:
def load_data():
  data = dict()
  for fn in os.listdir("stsbenchmark"):
    if fn.endswith(".csv"):
      with open("stsbenchmark/" + fn) as f:
        subset = fn[:-4].split("-")[1]
        #print(subset)
        data[subset] = dict()
        data[subset]['data'] = []
        data[subset]['scores'] = []
        for l in f:
          #genre filename year score sentence1 sentence2 (and sources, sometimes)
          l = l.strip().split("\t")
          data[subset]['data'].append((l[5],l[6]))
          data[subset]['scores'].append(float(l[4]))
  return data

dataset = load_data()

train_y = np.array(dataset['train']['scores']) / 5
dev_y = np.array(dataset['dev']['scores']) / 5
test_y = np.array(dataset['test']['scores']) / 5

### Having a look at the data...

#print("Some examples from the dataset:")
#for i in range(5):
#  print("s1:", dataset['train']['data'][i][0])
#  print("s2:", dataset['train']['data'][i][1])
#  print("score:", dataset['train']['scores'][i], "\n")
#
#print("\nNumber of sentence pairs by subset:")
#for subset in dataset:
#  print(subset, len(dataset[subset]['data']))
#
#print("\nRange of scores in the training set:", min(dataset["train"]["scores"]), "-", max(dataset["train"]["scores"]))

In [ ]:
def evaluate(predictions, gold_standard):
  return pearsonr(predictions, gold_standard)[0]

# Partie 1 : Simple features


## Preprocessing
---

In [ ]:
# Collective code
# Initialize resources
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def remove_non_ascii(input_string):
    input_string = unidecode(input_string)
    return re.sub(r'[^\x00-\x7F]+', '', input_string)


In [ ]:
# Collective code
def default_preprocess_sentence(sentence,
                                remove_punctuation=True,
                                use_stemming=True,
                                use_lemmatization=False,
                                remASCII=False,
                                remove_stopwords=False):
    # Tokenize and lowercase
    tokens = word_tokenize(sentence.lower())

    # Remove punctuation
    if remove_punctuation:
        tokens = [word for word in tokens if word not in string.punctuation]

    # Remove stopwords
    if remove_stopwords:
        tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    if use_stemming:
        tokens = [stemmer.stem(word) for word in tokens]
    elif use_lemmatization:
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Remove non ASCII
    if remASCII:
        tokens = [remove_non_ascii(word) for word in tokens]

    return tokens



In [ ]:
# Cell by Frédéric Lauron
def rm_unknow_wordnet(tokens):
  '''
  This function remove from the token list all words unknown in wordnet
  return a token list
  '''
  only_recognized_words = []
  for t in tokens:
    if wn.synsets(t[0]):
      only_recognized_words.append(t)
  tokens=only_recognized_words
  return tokens

def fromLTupleToLLis(s):
  '''
  this function take a tuple as argument copy it's content in a list and return the list
  '''
  res=[]
  for i in s:
    res.append(list(i))
  return res

def lemmatisation(s):
  res=s
  for i in res:
    if i[1]=='NOUN':
      i[0]=WordNetLemmatizer().lemmatize(i[0],pos='n')
    elif i[1]=='ADJ':
      i[0]=WordNetLemmatizer().lemmatize(i[0],pos='a')
    elif i[1]=='ADJ_SAT':
      i[0]=WordNetLemmatizer().lemmatize(i[0],pos='s')
    elif i[1]=='VERB':
      i[0]=WordNetLemmatizer().lemmatize(i[0],pos='v')
    elif i[1]=='ADV':
      i[0]=WordNetLemmatizer().lemmatize(i[0],pos='r')
  return res

def wordnet_preprocess_sentence(sentence):
  '''
  this function compute a preprocess, specific to wordnet feature
  '''
  #lowercase
  S_lower = sentence.lower()

  #tokenise
  S_lower_tok=word_tokenize(S_lower)

  #pos_tag
  S_lower_tok_tag = pos_tag(S_lower_tok,tagset="universal")

  #Create a list of list otherwise we manage tuple and we can't modify them
  S_ltt_list= fromLTupleToLLis(S_lower_tok_tag)

  #lemmatization
  S_ltt_list=lemmatisation(S_ltt_list)

  #remove words that are not in wordnet
  S_final = rm_unknow_wordnet(S_ltt_list)

  return S_final


In [ ]:
# Collective Code
def preprocess_dataset(dataset,
                       remove_punctuation=True,
                       use_stemming=False,
                       use_lemmatization=True,
                       remove_stopwords=True,
                       remASCII = False):
    preprocessed_dataset = {}
    wordnet_preprocessed_dataset = {}
    for subset, data in dataset.items():
      preprocessed_dataset[subset] = {'data': [], 'scores': data['scores']}
      wordnet_preprocessed_dataset[subset] = {'data': [], 'scores': data['scores']}
      for s1, s2 in data['data']:
        preprocessed_dataset[subset]['data'].append((default_preprocess_sentence(s1,
                                                                                 remove_punctuation=remove_punctuation,
                                                                                 remove_stopwords=remove_stopwords,
                                                                                 use_stemming=use_stemming,
                                                                                 use_lemmatization=use_lemmatization,
                                                                                 remASCII=remASCII),
                                                      default_preprocess_sentence(s2,
                                                                                 remove_stopwords=remove_stopwords,
                                                                                 remove_punctuation=remove_punctuation,
                                                                                 use_stemming=use_stemming,
                                                                                 use_lemmatization=use_lemmatization,
                                                                                 remASCII=remASCII)))
        wordnet_preprocessed_dataset[subset]['data'].append((wordnet_preprocess_sentence(s1),
                                                    wordnet_preprocess_sentence(s2)))

    return preprocessed_dataset,wordnet_preprocessed_dataset



In [ ]:
# Collective Code

default_preprocessed,wordnet_preprocessed = preprocess_dataset(dataset=dataset,
                                                               remove_punctuation=True,
                                                               use_stemming=False,
                                                               use_lemmatization=True,
                                                               remove_stopwords=True,
                                                               remASCII=False)

In [ ]:
wordnet_preprocessed["train"]["data"]

[([['a', 'DET'],
   ['plane', 'NOUN'],
   ['be', 'VERB'],
   ['take', 'VERB'],
   ['off', 'PRT']],
  [['an', 'DET'],
   ['air', 'NOUN'],
   ['plane', 'NOUN'],
   ['be', 'VERB'],
   ['take', 'VERB'],
   ['off', 'PRT']]),
 ([['a', 'DET'],
   ['man', 'NOUN'],
   ['be', 'VERB'],
   ['play', 'VERB'],
   ['a', 'DET'],
   ['large', 'ADJ'],
   ['flute', 'NOUN']],
  [['a', 'DET'],
   ['man', 'NOUN'],
   ['be', 'VERB'],
   ['play', 'VERB'],
   ['a', 'DET'],
   ['flute', 'NOUN']]),
 ([['a', 'DET'],
   ['man', 'NOUN'],
   ['be', 'VERB'],
   ['spread', 'VERB'],
   ['shred', 'VERB'],
   ['cheese', 'NOUN'],
   ['on', 'ADP'],
   ['a', 'DET'],
   ['pizza', 'NOUN']],
  [['a', 'DET'],
   ['man', 'NOUN'],
   ['be', 'VERB'],
   ['spread', 'VERB'],
   ['shred', 'VERB'],
   ['cheese', 'NOUN'],
   ['on', 'ADP'],
   ['an', 'DET'],
   ['uncooked', 'ADJ'],
   ['pizza', 'NOUN']]),
 ([['three', 'NUM'],
   ['men', 'NOUN'],
   ['be', 'VERB'],
   ['play', 'VERB'],
   ['chess', 'NOUN']],
  [['two', 'NUM'],
   ['men', 

## Feature extraction
---



In [ ]:
########################## Feature extraction functions ########################

# Function by Pierre DLVM
def get_ngram_overlap(s1,s2,n=2):
    # We check that we don't have a sentence to small to become an ngram, which would break the process
    if(len(s1.split()) < n or len(s2.split()) <n):
      if(s1 == s2):
        return 1
      else:
        return 0

    try:
        # We use a special pattern for token_pattern in order to avoid problems with tokens like 's
        cv = CountVectorizer(binary=True, ngram_range=(n, n),token_pattern=r"\b\w+\b|n't")
        vectors = cv.fit_transform([s1, s2]).toarray()
        # We can count the cardinal of the intersection through a simple dot product
        inter_cardinal = np.dot(vectors[0], vectors[1])
        if(inter_cardinal == 0):
          return 0
        s1overlap = np.sum(vectors[0])/inter_cardinal
        s2overlap = np.sum(vectors[1])/inter_cardinal
        if((s1overlap  + s2overlap) == 0):
            return 0
          #print("S1 inter S2 = ", inter_cardinal)
          #print(vectors)
          #print(cv.get_feature_names_out())
        ngo = 2*(1/(s1overlap + s2overlap))
    except:
        print("Ngram_overlap failed with the following pair of sentence")
        print(s1,"|||",s2)
        return 0
    return ngo

# Function by Marc Edouard
def tfidf_similarity(s1, s2, vectorizer):
    # Only fitting has been applied on 'train' subset, need to transform for all sentence pairs
    tfidf_matrix = vectorizer.transform([s1, s2])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

# Function by Yannick Letort
def get_bleu_score(s1,s2,weights=[1]):
  score1 = sentence_bleu([s1], s2,weights=weights)
  score2 = sentence_bleu([s2], s1,weights=weights)
  return harmonic_mean((score1,score2))



# Function by Pierre DLVM
def generate_ics(subset_data):
    data = []
    for pair in subset_data:
        data.extend([" ".join(sentence) for sentence in pair])
    # First we need to calculate the information content for each word
    cv = CountVectorizer(binary=True)#,token_pattern=r"\b\w+\b|n't")
    dataVectors = cv.fit_transform(data).toarray()
    names = cv.get_feature_names_out()
    freq_per_word = {}
    freqs = np.sum(dataVectors,axis=0)
    for i in range(len(names)):
        freq_per_word[names[i]] = freqs[i]

    ics = {}
    for name in names:
        ics[name] = math.log(np.sum(freqs)/freq_per_word[name])



    cv_tokenizer = cv.build_tokenizer()

    return ics,cv_tokenizer

# Function by Pierre DLVM
def get_weighted_word_overlap(s1,s2,ics,tokenizer):


    s1_tokenized = tokenizer(" ".join(s1))
    s2_tokenized = tokenizer(" ".join(s2))
    intersection = [token for token in s1_tokenized if token in s2_tokenized]
    # Make sure s1 and s2 both contain at least one word
    if(len(s1_tokenized) == 0 or len(s2_tokenized) == 0):
        return 0
    else:
        wwc12 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s1_tokenized)
        wwc21 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s2_tokenized)

    return harmonic_mean((wwc12,wwc21))


# Test
#reference = preprocessed_dataset['train']['data'][0][0]
#candidate = preprocessed_dataset['train']['data'][0][1]
#print(preprocessed_dataset['train']['data'][0])
#sentence_bleu([['This', 'is', 'test']],['This', 'is', 'a', 'cat'],weights = [1])
#print(get_bleu_score(preprocessed_dataset['train']['data'][0][0],preprocessed_dataset['train']['data'][0][1]))

In [ ]:
# Cell code by Frédéric Lauron

######################### Wordnet augmented overlap feature extraction function #################################
######################### Path Similarity AND Wu-Palmer Similarity ##############################################
def extract_wordnet_format(w):

    e=wn.synsets(w[0])
    wn_format = []

    for k in e:
        if w[1] == 'VERB' and k.pos() == 'v':
            wn_format.append(k.name())
        elif w[1] == 'ADJ' and (k.pos() == 'a' or k.pos() == 's' ):
            wn_format.append(k.name())
        elif w[1] == 'ADJ_SAT' and k.pos() == 's':
            wn_format.append(k.name())
        elif w[1] == 'ADV' and k.pos() == 'r':
            wn_format.append(k.name())
        elif w[1] == 'NOUN' and k.pos() == 'n':
            wn_format.append(k.name())
    return wn_format

def sim(w1,w2,similarity):
    w1_synset_list = extract_wordnet_format(w1)
    w2_synset_list = extract_wordnet_format(w2)
    score=[]

    #if one of the list of synsets is empty
    if (not w1_synset_list) or (not w2_synset_list):
        return 0.0

    for i in w1_synset_list:
        for j in w2_synset_list:
            if(similarity=="path"):
              score.append(wn.path_similarity(wn.synset(i), wn.synset(j)))#compute wordenet sysnsets take the max
            if(similarity=="wup"):
              score.append(wn.wup_similarity(wn.synset(i), wn.synset(j)))#compute wordenet sysnsets take the max
    return np.max(score)

def score (w,S,similarity):

    sc = []
    for i in S:
        if(w[0]==i[0]):#le mot w est dans S
            return 1
        sc.append(sim(w,i,similarity))

    return np.max(sc)

def WordNet_augmented_covrage(S1,S2,similarity):

    final_score = 0
    for i in S1:
        final_score += score(i,S2,similarity)

    final_score = final_score/len(S2)
    return final_score

def WordNet_augmented_word_overlap(S,T,similarity):
    '''
    Compute WordNet_augmented_word_overlap base on sentences S and T and the  similarity use ( path_similarity: "path" or wup_similarity: "wup")
    '''
    p1 = WordNet_augmented_covrage(S,T,similarity)
    p2 = WordNet_augmented_covrage(T,S,similarity)
    return harmonic_mean((p1,p2))

In [ ]:
# Collective code

# Fonction de feature extraction, prend les résultats du preprocessing + les features qu'on veut appliquer
def extract_features(subset_data,wordnet_data=None,bleu_score=True,wordnet1=True,wordnet2=True,ngram_overlap=True,TFIDF=True,weighted_word_overlap=True):
    features = []

    # Compute TF-IDF matrix for all data to be efficient
    tfidf_vectorizer = TfidfVectorizer()
    train_sentences = []

    for pair in subset_data:
        for i in range(2):
            sentence = " ".join(pair[i])
            #print('sentence - pair ', pair, i, sentence)
            train_sentences.append(sentence)
    tfidf_vectorizer.fit(train_sentences)
    ics, cv_tokenizer = generate_ics(subset_data=subset_data)

    for subset,wordnet in zip(subset_data,wordnet_data):
      s1_tokens = subset[0]
      s2_tokens = subset[1]
      s1, s2 = " ".join(s1_tokens), " ".join(s2_tokens)
      current_features = []

      # ngram
      current_features.append(get_ngram_overlap(s1,s2))
      # TFIDF similarity
      current_features.append(tfidf_similarity(s1,s2,tfidf_vectorizer))
      # Bleu score
      current_features.append(get_bleu_score(s1_tokens,s2_tokens))
      # Weighted word overlap
      current_features.append(get_weighted_word_overlap(s1_tokens,s2_tokens,ics,tokenizer=cv_tokenizer))

      s1_tokens_wn = wordnet[0]
      s2_tokens_wn = wordnet[1]
      # WordNet_augmented_word_overlap path similarity
      current_features.append(WordNet_augmented_word_overlap(s1_tokens_wn,s2_tokens_wn,"path"))
      # WordNet_augmented_word_overlap wu similarity
      current_features.append(WordNet_augmented_word_overlap(s1_tokens_wn,s2_tokens_wn,"wup"))

      features.append(current_features)

    return np.array(features)

# retourn une matrice X donc chaque colonne rpz une feature pour tous les échantillons

In [ ]:
train_features = extract_features(default_preprocessed["train"]["data"],wordnet_preprocessed["train"]["data"])

<ipython-input-16-66f2cc459103>:83: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  wwc12 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s1_tokenized)
<ipython-input-16-66f2cc459103>:84: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  wwc21 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s2_tokenized)
/usr/lib/python3.10/statistics.py:428: RuntimeWarning: divide by zero encountered in double_scalars
  T, total, count = _sum(w / x if w else 0 for w, x in zip(weights, data))


In [ ]:
dev_features = extract_features(default_preprocessed["dev"]["data"],wordnet_preprocessed["dev"]["data"])

<ipython-input-16-66f2cc459103>:83: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  wwc12 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s1_tokenized)
<ipython-input-16-66f2cc459103>:84: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  wwc21 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s2_tokenized)


In [ ]:
test_features = extract_features(default_preprocessed["test"]["data"],wordnet_preprocessed["test"]["data"])

<ipython-input-16-66f2cc459103>:83: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  wwc12 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s1_tokenized)
<ipython-input-16-66f2cc459103>:84: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  wwc21 = np.sum(ics[token] for token in intersection)/np.sum(ics[token] for token in s2_tokenized)


## Model building
---

In [ ]:
# Cell code by Yannick Letort
model_types = ["linear","random_forest","ridge","lasso","elastic_net","svm", "xgboost"]
def build_model(X,Y,type="linear"):
    if(type not in model_types):
        raise Exception("type needs to be a part of these : "+str(model_types))
    if type == "linear":
        model = LinearRegression()
    elif type == "random_forest":
        model = RandomForestRegressor()
    elif type == "ridge":
        model = RidgeCV()
    elif type == "lasso":
        model = LassoCV()
    elif type == "elastic_net":
        model = ElasticNetCV()
    elif(type in ["svm","xgboost"]):
        return build_and_tune_model(X,Y,type)



    model.fit(X,Y)
    return model




param_grid_svm = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf']}
''' # param_grid_svm = {'svr__C': [0.1, 1, 10, 100], 'svr__gamma': ['scale', 'auto'], 'svr__kernel': ['linear', 'rbf', 'poly']}
    param_grid_svm = {
    'svr__C': np.logspace(-2, 10, 13),
    'svr__gamma': np.logspace(-9, 3, 13),
    'svr__kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'svr__degree': [2, 3, 4]  # Uniquement pertinent pour le noyau 'poly'
}
'''
param_grid_xgb = {
    'xgboost__n_estimators': [100, 200, 300],
    'xgboost__learning_rate': [0.01, 0.05, 0.1],
    'xgboost__max_depth': [3, 5, 7],
    'xgboost__colsample_bytree': [0.7, 1],
    'xgboost__subsample': [0.6, 0.8, 1.0],
    'xgboost__gamma': [0, 0.1, 0.2]
}

# Fonction pour construire et ajuster les modèles
# Exhaustive search over specified parameter values for an estimator.
# Important members are fit, predict.
# GridSearchCV implements a “fit” and a “score” method. It also implements “score_samples”, “predict”, “predict_proba”.

#The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.
def build_and_tune_model(X, Y, type):
    if type == "svm":
        model = SVR()
        # times of compute important with GridSearchCV(). We'll use RandomizedSearch() to control the number of iteration
        # grid_search = RandomizedSearchCV(model, param_grid_svm, n_iter=5, cv=5, scoring='r2', random_state=42)
        grid_search = GridSearchCV(model, param_grid_svm, cv=5, scoring='r2')
    elif type == "xgboost":
        model = Pipeline([('scaler', StandardScaler()), ('xgboost',XGBRegressor(objective ='reg:squarederror'))])
        grid_search = RandomizedSearchCV(model, param_grid_xgb, n_iter=10, cv=5, scoring='r2', random_state=42)

    grid_search.fit(X, Y)
    return grid_search.best_estimator_



In [ ]:
# Cell code by Pierre DlVM

#features_train = np.load("features_train.npy")
#features_dev = np.load("features_dev.npy")
#features_test = np.load("features_test.npy")

features_train = train_features
features_dev = dev_features
features_test = test_features

features=["ngram overlap","TFIDF similarity","bleu score","weighted word overlap","wordnet overlap path","wordnet overlap wup"]

df_train_x = pd.DataFrame(np.array(features_train),columns=features)
df_dev_x = pd.DataFrame(np.array(features_dev),columns=features)
df_test_x = pd.DataFrame(np.array(features_test),columns=features)

separate_train_features = {}
for column in df_train_x.columns:
    separate_train_features[column] = df_train_x[column].values.reshape((-1,1))
separate_dev_features = {}
for column in df_dev_x.columns:
    separate_dev_features[column] = df_dev_x[column].values.reshape((-1,1))

separate_test_features = {}
for column in df_test_x.columns:
    separate_test_features[column] = df_test_x[column].values.reshape((-1,1))


In [ ]:
# Cell code by Pierre DlVM

def test_model_type(model_type):
    best_packages_per_combination = {}


    for j in range(1,len(features)+1):
        best_package = None
        for combination in list(list(combinations(features,j))):

            # Create feature matrix
            X_matrix = separate_train_features[combination[0]]
            X_testmatrix = separate_test_features[combination[0]]
            X_devmatrix = separate_dev_features[combination[0]]
            for t in range(1,len(combination)):
                X_matrix = np.column_stack((X_matrix,separate_train_features[combination[t]]))
                X_testmatrix = np.column_stack((X_testmatrix,separate_test_features[combination[t]]))
                X_devmatrix = np.column_stack((X_devmatrix,separate_dev_features[combination[t]]))

            # Now use the matrix to train the model
            current_model = build_model(X_matrix,train_y,type=model_type)
            current_package = [model_type,combination,evaluate(current_model.predict(X_devmatrix),dev_y),evaluate(current_model.predict(X_testmatrix),test_y)]
            print("Finished model")
            print(combination)

            if(best_package is None):
                best_package = current_package
            elif(best_package[2] < current_package[2]):
                best_package = current_package
        best_packages_per_combination[j] = best_package
    np.save("best_packages_"+model_type+".npy",best_packages_per_combination)
    print("Finished testing model :",model_type)
    return best_packages_per_combination

In [ ]:
# Cell code by Pierre DlVM

results = {}
for mtype in model_types:

    results[mtype] = test_model_type(mtype)

np.save("fullresults_part1.npy",results)

In [ ]:
print(results)

---

# Part 2 : Models relying on distributional and distributed word representation

## Distributional representation
(section done by Marc Edouard)

In [ ]:
#### Building distributional representations
sws = stopwords.words('english')
sws = set(list(sws) + [p for p in punctuation])

def put_sentences_together(dataset):
  all_sentences = []
  for s1, s2 in dataset["train"]["data"]:
    all_sentences.extend([s1, s2])
  return all_sentences


def create_vocabulary(dataset, count_threshold=1, voc_threshold=None, stopwords=set(), lowercase=False, use_lemmatization=True):
    """
    Function using word counts to build a vocabulary
    Params:
        corpus (list of list of strings): corpus of sentences
        count_threshold (int): minimum number of occurences necessary for a word to be included in the vocabulary
        voc_threshold (int): maximum size of the vocabulary
        stopwords: a set of words which are excluded from the vocabulary
        lowercase: bool. If True, all words are lowercased (which results in a smaller, more compact vocabulary)
    IMPORTANT: the vocabulary includes "UNK", which is a placeholder for an unknown word and it will later be assigned a zero vector.
    Returns:
        vocabulary (dictionary): keys: list of distinct words across the corpus
                                 values: indexes corresponding to each word sorted by frequency
    """
    corpus = put_sentences_together(dataset)
    word_counts = {}
    for sent in corpus:
        for word in word_tokenize(sent):
            if lowercase:
              word = word.lower()
            if use_lemmatization:
              word = lemmatizer.lemmatize(word)
            if word not in word_counts:
                word_counts[word] = 0
            word_counts[word] += 1
    # Create a dictionary called <filtered_word_counts> (with words as keys and their frequencies as values).
    # Include only those words that appear more than <count_threshold> times,
    # and which are not in the set of stopwords.
    ## TO COMPLETE

    filtered_word_counts = {word: count for word, count in word_counts.items() if count > count_threshold and word not in stopwords}

    # Create a list called <words> sorting the words from highest to lowest frequency
    ## TO COMPLETE
    words = sorted(filtered_word_counts, key=filtered_word_counts.get, reverse=True)

    ## ALREADY PRESENT
    if voc_threshold is not None:
        words = words[:voc_threshold] + ['UNK']
    #words = words  #Some word are removed with thd or stopword, so does make sense to add UNK in all cases
    vocabulary = {words[i] : i for i in range(len(words))}
    return vocabulary, {word: filtered_word_counts.get(word, 0) for word in vocabulary}

def co_occurence_matrix(dataset, vocab, lowercase=False):
    """
    Params:
        dataset: output of load_data()
        vocab: first output of create_vocabulary(). These are the words that will be included in the matrix
    Returns:
        matrix (array of size (len(vocab), len(vocab))): the co-occurrence matrix, using the same ordering as the vocabulary given in input
    """
    l = len(vocab)
    all_sentences = put_sentences_together(dataset)
    M = np.zeros((l,l))
    for sent in all_sentences:
        sent_idcs = []
        for word in word_tokenize(sent):
          if lowercase:
            word = word.lower()
          sent_idcs.append(vocab.get(word, len(vocab)-1))
        for i, idx in enumerate(sent_idcs):
            for j, ctx_idx in enumerate(sent_idcs[i+1:]):
              M[idx][ctx_idx] +=1
              M[ctx_idx][idx] +=1
    return M


def SentCheckWithVocab(dataset, vocab, NbSent):
    """
    Params:
        dataset: output of load_data()
        vocab: first output of create_vocabulary(). These are the words that will be included in the matrix
        NbSent: number of sentences to show
    Returns:
        None
    """
    l = len(vocab)
    all_sentences = put_sentences_together(dataset)
    M = np.zeros((l,l))
    index_to_word = {index: word for word, index in vocab.items()}
    id = 0
    for sent in all_sentences:
      print("Sentence: ", sent)
      sent_idcs = []
      sent_words = []
      for word in word_tokenize(sent):
        word = word.lower()
        idx = vocab.get(word, vocab.get('UNK'))
        sent_idcs.append(idx)
        sent_words.append(index_to_word.get(idx, 'UNK'))
      print((sent_idcs, sent_words))
      if id >= NbSent - 1:
        return None
      else :
        id = id + 1

In [ ]:
# Build vocabulary to check functions implementation  - here 2 vocab with only option changing

vocab_prepro, vocab_prepro_freq = create_vocabulary(dataset, count_threshold=1, voc_threshold=None, stopwords=sws, lowercase=True, use_lemmatization=False)
vocab, vocab_freq = create_vocabulary(dataset, count_threshold=1, voc_threshold=None, stopwords=sws, lowercase=False, use_lemmatization=False)
print('Taille du vocabulaire: ', len(vocab))
print(vocab_freq)
print('Taille du vocabulaire: ', len(vocab_prepro))
print(vocab_prepro_freq)

# Build co-occurrence matrix
M = co_occurence_matrix(dataset, vocab, True)
M_prepro = co_occurence_matrix(dataset, vocab_prepro)

print("Taille des matrices de co-occurrences:", M.shape, M_prepro.shape)

In [ ]:
# Use SentCheckWithVocab to check on some sample of sentences impact of pre_processing
# Remind last argument is number of sentences to sample

SentCheckWithVocab(dataset, vocab, 10)
print("\n #### With too restrictive preprocessing")
SentCheckWithVocab(dataset, vocab_prepro, 10)

In [ ]:
## Check repartition of occurences

occurrences = list(vocab_freq.values())
occurrences_lem = list(vocab_prepro_freq.values())

# Plot a histogram
plt.figure(figsize=(6, 2))
plt.hist(occurrences, bins=range(1, max(occurrences)+1), log=True)
plt.hist(occurrences_lem, bins=range(1, max(occurrences)+1), log=True)
plt.title('Histogram of Word Occurrences')
plt.xlabel('Occurrences')
plt.ylabel('Number of words')
# Set the scale of the x-axis to logarithmic to better display a wide range of values
plt.xscale('log')
plt.show()

In [ ]:
# As the preprocessing has several option to be configure, it could be interresting to analyze which options
# are impacting the size of the vocabulary. The objective is to discern to which parameters the size of the
# vocabulary is sensitive, and ensure one's shrinking it to very small vocab (possibility to miss lot of
# information)

threshold_values = [0, 1, 2, 3, 5, 8, 13, 21]

configurations = [
    {'use_lemmatization': True, 'stopwords': sws, 'lowercase': True},
    {'use_lemmatization': True, 'stopwords': set(), 'lowercase': True},
    {'use_lemmatization': False, 'stopwords': sws, 'lowercase': True},
    {'use_lemmatization': False, 'stopwords': set(), 'lowercase': True},
    {'use_lemmatization': True, 'stopwords': sws, 'lowercase': False},
    {'use_lemmatization': True, 'stopwords': set(), 'lowercase': False},
    {'use_lemmatization': False, 'stopwords': sws, 'lowercase': False},
    {'use_lemmatization': False, 'stopwords': set(), 'lowercase': False}
]

# Initialize dictionaries to hold the results
vocab_lengths = {config: [] for config in range(len(configurations))}
sentence_word_counts = {config: [] for config in range(
    len(configurations) * len(threshold_values))}

# Loop over each configuration and threshold
for config_idx, config in enumerate(configurations):
    for count_threshold in threshold_values:

        idx = config_idx + len(configurations) * \
            threshold_values.index(count_threshold)

        # Build vocabulary for current config and threshold
        vocab, vocab_freq = create_vocabulary(dataset, count_threshold=count_threshold,
                                        stopwords=config['stopwords'], lowercase=True,
                                        use_lemmatization=config['use_lemmatization'])
        vocab_lengths[config_idx].append(len(vocab))

        # Check sentences "smashed" by preprocessing
        all_sentences = put_sentences_together(dataset)
        for sent in all_sentences:
            words = sent.lower().split()
            words_in_vocab = sum(word in vocab for word in words)
            sentence_word_counts[idx].append(words_in_vocab)

def get_config_label(config):
    lemmatization = "Lem = T" if config['use_lemmatization'] else "Lem = F"
    stopwords = "With Sws = T" if bool(config['stopwords']) else "With Sws = F"
    lowercase = "With lwr = T" if config['lowercase'] else "With lwr = F"
    return f"{lemmatization}, {stopwords}, {lowercase}"

# Plotting vocabulary size for each configuration
plt.figure(figsize=(12, 5))
for config_idx, config in enumerate(configurations):
    config_label = get_config_label(config)
    plt.plot(threshold_values,
             vocab_lengths[config_idx], marker='o', label=f"{config_label}")
plt.title('Vocabulary Size vs. Count Threshold')
plt.xlabel('Count Threshold')
plt.ylabel('Vocabulary Size')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# A function that converts the matrix values to PMI
def pmi(M, positive=True):
    eps = 1e-8

    sum_vec = M.sum(axis=0) + eps
    sum_tot = sum_vec.sum()
    with np.errstate(divide='ignore'):
        pmi = np.log((M * sum_tot) / (np.outer(sum_vec, sum_vec)))
    pmi[np.isinf(pmi)] = 0.0  # log(0) = 0
    if positive:
        pmi[pmi < 0] = 0.0
    return pmi

# To apply the transformation:
PMI_M = pmi(M)

In [ ]:
# Loading the postagger
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')

Nb_Debug = 10


"""Tokenize the sentences.
Optionally lowercase the tokens.
Optionally apply POS tagging and filter based on specific POS tags.
Retrieve the distributional vectors for each word in the sentences.
Aggregate the word vectors to obtain a single vector representation for each sentence.
Compute the desired features (either similarity/distance or concatenated vectors)."""

def normalize_vector(vec):
    '''Normalize a vector to unit length'''
    norm = np.linalg.norm(vec)
    if norm == 0:
       return vec
    return vec / norm

def kl_divergence(p, q):
    ''' Calculate KL Divergence for two probability distributions '''
    p = np.clip(p, 1e-10, 1)
    q = np.clip(q, 1e-10, 1)
    return entropy(p, q)

def assign_distributional_vectors(data, M, vocab, sim_or_dist=True, postag=False, lowercase=False, Debug=False, Dist = "Cosinus"):
  '''This functions assigns each sentence a vector and optionally calculates the similarity/distance
  between the representations of s1 and s2.
  Parameters
    data: list of tuples (like dataset['train']['data'])
    M: a matrix of distributional representations for all words in the vocabulary
    vocab: first output of create_vocabulary(). These are the words that will be included in the matrix
    sim_or_dist: bool. If True, we will use a similarity or distance as the only feature. If False,
    we will use the concatenation of the representations of s1 and s2.
    postag: whether we want to apply a postag-based filter to obtain sentence representations
    lowercase: bool. If True, words are lowercased. You should set it to True if the vocabulary is lowercased.
  Returns:
    features: an array with the data transformed into features '''
  Nb_Novec = 0

  if sim_or_dist:
    features = np.zeros((len(data), 1))
    #features = np.zeros((len(data), 3))
  else:
    features = np.zeros((len(data), M.shape[1]*2))
  for i, (s1, s2) in enumerate(data):
    # Tokenize, lowercase if lowercase=True, and if postag=True, postag s1 and s2
    ## TO COMPLETE
   tokens1 = word_tokenize(s1)
   tokens2 = word_tokenize(s2)

   if lowercase:
         tokens1 = [token.lower() for token in tokens1]
         tokens2 = [token.lower() for token in tokens2]

   # Filter out all token that are not "noun", "verb", "adj", "adv"
   if postag:
         tokens1 = [word for word, pos in pos_tag(tokens1, tagset='universal') if pos in ['NOUN', 'VERB', 'ADJ', 'ADV']]
         tokens2 = [word for word, pos in pos_tag(tokens2, tagset='universal') if pos in ['NOUN', 'VERB', 'ADJ', 'ADV']]

   if i <= Nb_Debug and Debug:
      print("postag s1:", [pos for word, pos in pos_tag(tokens1, tagset='universal')])
      print("postag s2:", [pos for word, pos in pos_tag(tokens2, tagset='universal')])
      print("Token: ", tokens1, tokens2)


    # Now create two lists, one for each sentence, with the word representations that you want to use
    # You can go through the words (or word, pos) in each sentence and decide whether you keep their representation or not
    ## TO COMPLETE

   s1vecs = [M[vocab[word]] for word in tokens1 if word in vocab] #unknown word leaved out
   s1vecs_raw = [M[vocab[word],:] for word in tokens1 if word in vocab]
   """if Debug and all(np.array_equal(vec1, vec2) for vec1, vec2 in zip(s1vecs, s1vecs_raw)):
      print("s1vecs and raw corresponds")"""

   s2vecs = [M[vocab[word]] for word in tokens2 if word in vocab] #unknown word leaved out

   # Each word vector is of length of |Vocabulary|
   if i <= Nb_Debug and Debug:
      print([(word, vocab[word]) for word in tokens1 if word in vocab])
      print([(word, vocab[word]) for word in tokens2 if word in vocab])
      #print("Svecs: ", len(s1vecs), len(s2vecs), "s1vecs :" , s1vecs, "s2vecs :", s2vecs)

    # It is possible that some sentences will not have any word representation available.
    # We assign them a 0-vector in this case (be careful, because this could result in a cosine of NaN)
   if not s1vecs:
      s1vecs = [np.zeros(M.shape[1])]
   if not s2vecs:
      s2vecs = [np.zeros(M.shape[1])]
   if not s1vecs or not s2vecs :
      Nb_Novec = Nb_Novec + 1

    # Aggregate the representations of words in a sentence, for example by averaging them
    ## TO COMPLETE

   s1vec = np.mean(s1vecs, axis=0)
   s2vec = np.mean(s2vecs, axis=0)

   """   if i <= Nb_Debug and Debug:
      print("Svec - shape for each vect :", s1vec.shape, s2vec.shape, "Vecs :", s1vec, s2vec)"""


    # Fill in features[i] with the desired feature (one or more similarity/distance measures if sim=True,
    # a concatenation of the representations otherwise)
    ## TO COMPLETE

   if sim_or_dist:
      if Dist == "Cosinus":
        # Cosine similarity as an example <<<<<
        dot_product = np.dot(s1vec, s2vec)
        norm_s1 = np.linalg.norm(s1vec)
        norm_s2 = np.linalg.norm(s2vec)
        if norm_s1 == 0 or norm_s2 == 0:
          features[i] = 0
        else:
          features[i] = dot_product / (norm_s1 * norm_s2)

      elif Dist == "KL":
         features[i] = kl_divergence(s1vec, s2vec)

      elif Dist == "Euclid":
        s1vec = normalize_vector(s1vec)
        s2vec = normalize_vector(s2vec)
        features[i] = euclidean(s1vec, s2vec)
      else :
        s1vec = normalize_vector(s1vec)
        s2vec = normalize_vector(s2vec)
        features[i] = cityblock(s1vec, s2vec)
      #features[i] = [cos_similarity, euclidean_distance, manhattan_distance]
   else:
      # Concatenate vectors
      s1vec = normalize_vector(s1vec)
      s2vec = normalize_vector(s2vec)
      features[i] = np.concatenate((s1vec, s2vec))

   if i <= Nb_Debug and Debug:
      print("Features - shape :", features[i].shape, "Vec:", features[i])

  if Debug :
    print("Sxvecs with no word rep :", Nb_Novec)

  return features


In [ ]:
###########################################################################################################
###########################################################################################################
################### WARNING : VERY LONG TO RUN ###########################################################

# Obtain train_x, dev_x, test_x
# Try different combinations (using M, PMI_M, different aggregation functions, with and without postag filtering...)
# TO COMPLETE

# Define your parameter grid
param_grid = {
    'use_pmi': [True, False],
    'Word_count': [1],
    'voc_thd': [None, 8000, 5000, 3000, 1000, 500, 300],
    'sim_or_dist': [True, False],
    'use_lower': [True, False],
    'use_postag': [True, False],
    'use_lem': [True, False],
    'dist': ["KL", "Cosinus", "Euclid", "Manh"]
}

param_grid_svm = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf']}

Results = []
# Loop over all combinations of parameters
for voc_thd in param_grid['voc_thd']:
    for Word_count in param_grid['Word_count']:
        for lower in param_grid['use_lower']:
            for use_lem in param_grid["use_lem"]:
                # Build your vocab and co-occurrence matrix with the current parameters
                vocab, vocab_freq = create_vocabulary(dataset, count_threshold=Word_count, voc_threshold=voc_thd,
                                                      stopwords=sws, lowercase=lower, use_lemmatization=use_lem)
                M = co_occurence_matrix(dataset, vocab)

                for use_pmi in param_grid['use_pmi']:
                    if use_pmi:
                        M = pmi(M, positive=True)

                    for use_postag in param_grid["use_postag"]:
                        for sim_or_dist in param_grid['sim_or_dist']:

                            for distance in param_grid['dist']:

                                Model_caract = f"PMI: {use_pmi}, Word_count: {Word_count}, voc_thd : {voc_thd} use_lower: {lower} Sim/Dist: {sim_or_dist}, use_postag: {use_postag}, use_lem : {use_lem}, dist : {distance}"

                                # Generate features
                                features_train = assign_distributional_vectors(
                                    dataset['train']['data'], M, vocab, sim_or_dist=sim_or_dist, postag=use_postag, Dist=distance)
                                features_dev = assign_distributional_vectors(
                                    dataset['dev']['data'], M, vocab, sim_or_dist=sim_or_dist, postag=use_postag, Dist=distance)

                                if np.isnan(M).any():
                                    print("NaNs in co-occurrence matrix",
                                          np.isnan(M).sum(), Model_caract)

                                # Check for NaN values in the training features
                                if np.isnan(features_train).any():
                                    print("NaNs in training features", np.isnan(
                                        features_train).sum(), Model_caract)

                                # Check for NaN values in the dev features
                                if np.isnan(features_dev).any():
                                    print("NaNs in dev features", np.isnan(
                                        features_dev).sum(), Model_caract)

                                # Train and evaluate your model using cross-validation
                                # For example, using a simple linear model:
                                model = LinearRegression()
                                model.fit(features_train, train_y)
                                train_y_pred = model.predict(features_train)
                                dev_y_pred = model.predict(features_dev)

                                print("LinReg - Train :", np.round(evaluate(train_y_pred, train_y), 3),
                                      "Dev: ", np.round(evaluate(dev_y_pred, dev_y), 3), Model_caract)

                                """modelSVR = SVR()
                                grid_search = GridSearchCV(
                                    modelSVR, param_grid_svm, cv=5, scoring='r2')
                                grid_search.fit(features_train, train_y)
                                train_y_pred_SVR = grid_search.predict(
                                    features_train)
                                dev_y_pred_SVR = grid_search.predict(
                                    features_dev)

                                print("SVR - Train :", np.round(evaluate(train_y_pred_SVR, train_y), 3),
                                      "Dev: ", np.round(evaluate(dev_y_pred_SVR, dev_y), 3), Model_caract)

                                if sim_or_dist == False:

                                    alphas = np.logspace(-6, 6, 30)
                                    Ridge_cv = RidgeCV(alphas=alphas, cv=5)
                                    Ridge_cv.fit(features_train, train_y)
                                    train_y_pred_ridge = Ridge_cv.predict(
                                        features_train)
                                    dev_y_pred_ridge = Ridge_cv.predict(
                                        features_dev)

                                    train_corr_ridge = np.corrcoef(
                                        train_y, train_y_pred_ridge)[0, 1]
                                    dev_corr_lasso = np.corrcoef(
                                        dev_y, dev_y_pred_ridge)[0, 1]

                                    print("Ridge - Train :", np.round(evaluate(train_y_pred_ridge, train_y),
                                          3), "Dev: ", np.round(evaluate(dev_y_pred_ridge, dev_y), 3))"""

In [ ]:
# As running above models is very long, more practical to load previous results
file_path = 'results_models.txt'

# Function to parse each line
def parse_line(line):
    # Extracting the train and dev scores using regular expressions
    train_score = re.search(r'Train : ([0-9.]+)', line)
    dev_score = re.search(r'Dev: *(-?[0-9.]+)', line)
    config = re.search(r'PMI:.*', line)
    return {
        'Train': float(train_score.group(1)) if train_score else None,
        'Dev': float(dev_score.group(1)) if dev_score else None,
        'Config': config.group(0) if config else None
    }

# Reading and parsing the file
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(parse_line(line))

# Convert to DataFrame
df = pd.DataFrame(data)
df.sort_values(by='Dev', ascending=False, inplace=True)
df['Idx'] = range(1, len(df) + 1)

# Plotting
plt.figure(figsize=(15, 4))
plt.plot(df['Idx'], df['Train'], label='Train Score',
         marker='o', linestyle='None')
plt.plot(df['Idx'], df['Dev'], label='Dev Score', marker='x', linestyle='None')
plt.xlim([0, 896])
plt.xlabel('Model configuration index')
plt.ylabel('Score')
plt.title('Train and Dev Scores')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Can visualize relation between feature and score on a specific model, say one with highest performance

vocab, vocab_freq = create_vocabulary(dataset, count_threshold=1, voc_threshold=5000,
                                                stopwords=sws, lowercase=False, use_lemmatization=False)
print("Vocab size: ", len(vocab))

M = co_occurence_matrix(dataset, vocab)

# Debug - Generate features with representation similarity & co-occurence matrix
features_train_M_Sim = assign_distributional_vectors(dataset['train']['data'], M, vocab, sim_or_dist=True, postag=True, Debug=False, Dist="Euclid")

# Debug - Generate features with representation similarity & PMI matrix
PMI_M = pmi(M)
features_train_PMI_Sim = assign_distributional_vectors(dataset['train']['data'], PMI_M, vocab, sim_or_dist=True, postag=True, Debug=False, Dist="Euclid")

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
axes[0].scatter(features_train_M_Sim, train_y)
axes[1].scatter(features_train_PMI_Sim, train_y)
axes[0].set_xlabel("Feature - Euclidian distance & M")
axes[1].set_xlabel("Feature - Euclidian distance & PMI")
axes[0].set_ylabel("Score")
plt.show()

In [ ]:
# In above result, we see that there is no evident correlation between the two and we can suspect that performances won't be
# perfect

# Evaluate the best model on the test set
## TO COMPLETE
vocab, vocab_freq = create_vocabulary(dataset, count_threshold=1, voc_threshold=None,
                                                        stopwords=sws, lowercase=False, use_lemmatization=False)
M = co_occurence_matrix(dataset, vocab)
M = pmi(M)

features_train = assign_distributional_vectors(
                                    dataset['train']['data'], M, vocab, sim_or_dist=True, postag=True, Dist="euclid")
features_test = assign_distributional_vectors(
                                    dataset['test']['data'], M, vocab, sim_or_dist=True, postag=True, Dist="euclid")

model = LinearRegression()
model.fit(features_train, train_y)
train_y_pred = model.predict(features_train)
test_y_pred = model.predict(features_test)

print("LinReg - Train :", np.round(evaluate(train_y_pred, train_y),3), "Test: ", np.round(evaluate(test_y_pred, test_y),3))

In [ ]:
# One clue for these performances can be the relevancy of the corpus used for vocabulary creation

def put_sentences_together_new(dataset, Subset):
  all_sentences = []
  for s1, s2 in dataset[Subset]["data"]:
    all_sentences.extend([s1, s2])
  return all_sentences


def compare_subset_vocabulary(dataset, count_threshold=1, voc_threshold=None, stopwords=set(), lowercase=False, use_lemmatization=False):
    """
    Function using word counts to build a vocabulary
    Params:
        corpus (list of list of strings): corpus of sentences
        count_threshold (int): minimum number of occurences necessary for a word to be included in the vocabulary
        voc_threshold (int): maximum size of the vocabulary
        stopwords: a set of words which are excluded from the vocabulary
        lowercase: bool. If True, all words are lowercased (which results in a smaller, more compact vocabulary)
    IMPORTANT: the vocabulary includes "UNK", which is a placeholder for an unknown word and it will later be assigned a zero vector.
    Returns:
        vocabulary (dictionary): keys: list of distinct words across the corpus
                                 values: indexes corresponding to each word sorted by frequency
    """
    Subsets = ["train", "dev", "test"]
    vocab = []
    for s in Subsets:
      corpus = put_sentences_together_new(dataset, s)
      word_counts = {}
      for sent in corpus:
         for word in word_tokenize(sent):
            if lowercase:
               word = word.lower()
            if use_lemmatization:
               word = lemmatizer.lemmatize(word)
            if word not in word_counts:
               word_counts[word] = 0
            word_counts[word] += 1
    # Create a dictionary called <filtered_word_counts> (with words as keys and their frequencies as values).
    # Include only those words that appear more than <count_threshold> times,
    # and which are not in the set of stopwords.
    ## TO COMPLETE

      filtered_word_counts = {word: count for word, count in word_counts.items() if count > count_threshold and word not in stopwords}

      vocab.append(filtered_word_counts)
    return vocab

vocab = compare_subset_vocabulary(dataset, count_threshold=1, voc_threshold=None, stopwords=set(), lowercase=False, use_lemmatization=True)
vocab_set_0 = set(vocab[0].keys())
vocab_set_1 = set(vocab[1].keys())
vocab_set_2 = set(vocab[2].keys())

common_words_0_1 = vocab_set_0.intersection(vocab_set_1)
common_words_0_2 = vocab_set_0.intersection(vocab_set_2)


total_unique_words_0_1 = len(vocab_set_0.union(vocab_set_1))
total_unique_words_0_2 = len(vocab_set_0.union(vocab_set_2))

ratio_common_0_1 = len(common_words_0_1) / total_unique_words_0_1
ratio_common_0_2 = len(common_words_0_2) / total_unique_words_0_2

print("Ratio of common words between vocab[0] and vocab[1]:", ratio_common_0_1)
print("Ratio of common words between vocab[0] and vocab[2]:", ratio_common_0_2)

## Distributed representation

In [ ]:
# This may take a while...
! pip install pymagnitude

  Using cached pymagnitude-0.1.143.tar.gz (5.4 MB)
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
^C


In [ ]:
### How pymagnitude works
#bug pymagnitutude
#lien vers une solution
#https://github.com/plasticityai/magnitude/issues/92
from pymagnitude import *

# Loading vectors
## TO COMPLETE : specify the path to the magnitude file in ""
vectors = Magnitude("GoogleNews-vectors-negative300.magnitude",normalized=False)

# This is how you obtain the vector for a word (for example, "cat"):
cat_vector = vectors.query("cat")

ModuleNotFoundError: ignored

In [ ]:
#! pip install pyemd
#! pip install gensim
from gensim.corpora.dictionary import Dictionary
from pyemd import emd

## A function that calculates Word Mover's Distance, slightly modified from https://tedboy.github.io/nlps/_modules/gensim/models/word2vec.html#Word2Vec.wmdistance

def wmdistance(words1, words2, vectors):
    """
    words1 and words2 are lists of str containing the words of the first and second sentence in a sentence pair.
    These should include only words that you are interested in (e.g., maybe you exclude stopwords or words of some pos)
    vectors is a Magnitude object (created above with the downloaded word embeddings)
    """
    # Remove out-of-vocabulary words.
    if len(words1) == 0 or len(words2) == 0:
        print('At least one of the documents had no words that were'
                    'in the vocabulary. Aborting (returning inf).')
        return float('inf')

    dictionary = Dictionary(documents=[words1, words2])
    vocab_len = len(dictionary)

    # Sets for faster look-up.
    docset1 = set(words1)
    docset2 = set(words2)

    # Compute distance matrix.
    distance_matrix = np.zeros((vocab_len, vocab_len)) #, dtype=double)
    for i, t1 in dictionary.items():
        for j, t2 in dictionary.items():
            if not t1 in docset1 or not t2 in docset2:
                continue
            # Compute Euclidean distance between word vectors.
            distance_matrix[i, j] = np.sqrt(np.sum((vectors.query(t1) - vectors.query(t2))**2))

    if np.sum(distance_matrix) == 0.0:
        # `emd` gets stuck if the distance matrix contains only zeros.
        print('The distance matrix is all zeros. Aborting (returning inf).')
        return float('inf')

    def nbow(words):
        d = np.zeros(vocab_len) #, dtype=double)
        nbow = dictionary.doc2bow(words)  # Word frequencies.
        doc_len = len(words)
        for idx, freq in nbow:
            d[idx] = freq / float(doc_len)  # Normalized word frequencies.
        return d

    # Compute nBOW representation of documents.
    d1 = nbow(words1)
    d2 = nbow(words2)

    # Compute WMD.
    return emd(d1, d2, distance_matrix)


wmdistance(["cat", "be","blue"],["dog","be","red"],vectors)

In [ ]:
def assign_distributed_vectors(data, vectors, type_features,agreg_func="mean", postag=False, lowercase=False):
  # Write a function like assign_distributional_vectors which uses distributed
  # vectors. You can also simply modify that function.
  '''This functions assigns each sentence a vector and optionally calculates the similarity/distance
  between the representations of s1 and s2.
  Parameters
    data: list of tuples (like dataset['train']['data'])
    vectors: vectors in Magnitude format
    sim_or_dist: bool. If True, we will use a similarity or distance as the only feature. If False,
    we will use the concatenation of the representations of s1 and s2.
    postag: whether we want to apply a postag-based filter to obtain sentence representations
    lowercase: bool. If True, words are lowercased.
    wmdistance: bool. If True, Word Mover's Distance is used.
  Returns:
    features: an array with the data transformed into features '''
  ## TO COMPLETE

  if type_features == "simdist" or type_features == "wmdist":
    features = np.zeros((len(data), 1))
  if type_features=="concatenate":
    features = np.zeros((len(data), vectors.dim*2))


  for i, (s1, s2) in enumerate(data):
  # Tokenize, lowercase if lowercase=True, and if postag=True, postag s1 and s2
  ## TO COMP LETE
    tokens1 = word_tokenize(s1)
    tokens2 = word_tokenize(s2)

    if lowercase:
      tokens1 = [token.lower() for token in tokens1]
      tokens2 = [token.lower() for token in tokens2]

   # Filter out all token that are not "noun", "verb", "adj", "adv"
    if postag:
      tokens1 = [word for word, pos in pos_tag(tokens1, tagset='universal') if pos in ['NOUN', 'VERB', 'ADJ', 'ADV']]
      tokens2 = [word for word, pos in pos_tag(tokens2, tagset='universal') if pos in ['NOUN', 'VERB', 'ADJ', 'ADV']]

    # Now create two lists, one for each sentence, with the word representations that you want to use
    # You can go through the words (or word, pos) in each sentence and decide whether you keep their representation or not
    ## TO COMPLETE
    s1vecs = [vectors.query(word) for word in tokens1 if word in vectors] #unknown word leaved out
    s2vecs = [vectors.query(word) for word in tokens2 if word in vectors] #unknown word leaved out

    # It is possible that some sentences will not have any word representation available.
    # We assign them a 0-vector in this case (be careful, because this could result in a cosine of NaN)
    if not s1vecs:
      s1vecs = [np.zeros(vectors.dim)]
    if not s2vecs:
      s2vecs = [np.zeros(vectors.dim)]

    # Aggregate the representations of words in a sentence, for example by averaging them
    ## TO COMPLETE
    if agreg_func=="mean":
      s1vec = np.mean(s1vecs, axis=0)
      s2vec = np.mean(s2vecs, axis=0)

    if agreg_func=="std":
      s1vec = np.std(s1vecs, axis=0)
      s2vec = np.std(s2vecs, axis=0)

    if agreg_func=="median":
      s1vec = np.median(s1vecs, axis=0)
      s2vec = np.median(s2vecs, axis=0)

    # Fill in features[i] with the desired feature (one or more similarity/distance measures if sim=True,
    # a concatenation of the representations otherwise)
    ## TO COMPLETE
    if type_features == "simdist":
      # Cosine similarity as an example <<<<<
      dot_product = np.dot(s1vec, s2vec)
      norm_s1 = np.linalg.norm(s1vec)
      norm_s2 = np.linalg.norm(s2vec)
      if norm_s1 == 0 or norm_s2 == 0:
        similarity = 0
      else:
        similarity = dot_product / (norm_s1 * norm_s2)
      features[i] = similarity

    if type_features == "concatenate":
      # Concatenate vectors
      s1vec = normalize_vector(s1vec)
      s2vec = normalize_vector(s2vec)
      features[i] = np.concatenate((s1vec, s2vec))

    if type_features == "wmdist":
      features[i]= wmdistance(tokens1, tokens2, vectors)

  return features

In [ ]:
# Obtain train_x, dev_x, test_x
# Just as before, try different combinations (different aggregation functions, with and without postag filtering, with and without WMDistance...)
## TO COMPLETE

from sklearn.linear_model import LinearRegression, RidgeCV, ElasticNetCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
# from sklearn.svm import SVR
# from xgboost import XGBRegressor


def build_model(X, Y, type="linear"):
    if (type not in model_types):
        raise Exception("type needs to be a part of these : "+str(model_types))
    if type == "linear":
        model = LinearRegression()
    elif type == "random_forest":
        model = RandomForestRegressor()
    elif type == "ridge":
        model = RidgeCV()
    elif type == "lasso":
        model = LassoCV()
    elif type == "elastic_net":
        model = ElasticNetCV()
    '''
    elif type == "svm":
        model = SVR()
    elif type == "xgboost":
        model = XGBRegressor(objective ='reg:squarederror')
    '''

    model.fit(X, Y)
    return model



for feature in ["wmdist"]:
    for agreg_func in["mean"]:
        for use_pos in [True,False]:
            for use_lower in [True,False]:
                for model_types in ["linear", "ridge", "lasso", "elastic_net"]:

                    features_train = assign_distributed_vectors(dataset['train']['data'], vectors, feature, agreg_func, postag=use_pos, lowercase=use_lower)
                    features_dev = assign_distributed_vectors(dataset['dev']['data'], vectors, feature, agreg_func, postag=use_pos, lowercase=use_lower)

                    Model_caract = f"feature: {feature}, agreg_func: {agreg_func}, use_postag: {use_pos}, use_lower: {use_lower},model_types:{model_types}:"
                    model = build_model(features_train, train_y, type=model_types)

                    # Evaluation sur le dev set
                    dev_y_pred = model.predict(features_dev)
                    Eval = evaluate(dev_y_pred, dev_y)

                    f = open("recap.txt", "a")
                    print(Model_caract)
                    f.write(Model_caract+"\n")

                    if model_types == "elastic_net" or model_types == "lasso" or model_types == "ridge":
                        print("alpha:",model.alpha_)
                        f.write("alpha:"+f"{model.alpha_}")
                        f.write("\n")
                    if model_types == "elastic_net":
                        print("ratio:", model.l1_ratio_)
                        f.write("ratio:"+f"{model.l1_ratio_}")
                        f.write("\n")
                    print(Eval)
                    f.write("Eval:"+f"{Eval}")
                    f.write("\n")
                    f.close()



In [ ]:
# Evaluate the best model on the test set

from sklearn.linear_model import Lasso

features_best = assign_distributed_vectors(
    dataset['train']['data'], vectors, "simdist", "mean", postag=True, lowercase=False)
features_test = assign_distributed_vectors(
    dataset['test']['data'], vectors, "simdist", "mean", postag=True, lowercase=False)

model = Lasso(0.00011586083566376204)
model.fit(features_best, train_y)

prediction_test_y = model.predict(features_test)

Eval = evaluate(prediction_test_y, test_y)
print(Eval)

# Part 3 : Testing our methods on a spanish dataset

In [ ]:
# Loading team A datasets
! unzip STS2015-es-test.zip -d STS2015-es-test
! unzip Sts2017.eval.v1.1.zip
! unzip Sts2017.gs.zip

In [ ]:
def load_mono_spanish_data():
  traindir = "STS2015-es-test/"
  data = dict()
  for subset in ["train","test"]:
    data[subset] = dict()
    data[subset]["data"] = []
    data[subset]["scores"] = []
  train_data_by_category = dict()
  train_scores_by_category = dict()
  for fn in os.listdir(traindir):
    _, typ, category, _ = fn.split(".")
    with open(traindir + fn) as f:
      if typ == "gs":
        train_scores_by_category[category] = []
        for l in f:
          train_scores_by_category[category].append(float(l))
      elif typ == "input":
        train_data_by_category[category] = []
        for l in f:
          train_data_by_category[category].append(l.strip().split("\t"))

  for cat in train_data_by_category:
    data['train']['data'].extend(train_data_by_category[cat])
    data['train']['scores'].extend(train_scores_by_category[cat])

  #### now test data and then we merge them...
  with open("STS2017.eval.v1.1/STS.input.track3.es-es.txt") as f:
    for l in f:
      data['test']['data'].append(l.strip().split("\t"))
  with open("STS2017.gs/STS.gs.track3.es-es.txt") as f:
    for l in f:
      data['test']['scores'].append(float(l))

  return data



In [ ]:
# Load the data

mono_data = load_mono_spanish_data()

print("Score ranges:")
print(min(mono_data['train']['scores']), "-", max(mono_data['train']['scores']))
print(min(mono_data['test']['scores']), "-", max(mono_data['test']['scores']))

## Careful! The range of scores is different in the Spanish training set. It goes from 0 to 4.
# We will map it to a 0-5 scale

mono_data['train']['scores'] = (np.array(mono_data['train']['scores']) / 4) * 5

## Also, you may notice that some sentence pairs are repeated within the training set, but have different scores.
## We will simply make sure that there is no overlap between the training and the test datasets:
train_y = mono_data['train']['scores']
test_y = mono_data['test']['scores']


found = False
tr_sps = [tuple(ins) for ins in mono_data['train']['data']]
te_sps = [tuple(ins) for ins in mono_data['test']['data']]
for ins in te_sps:
  if ins in tr_sps:
    found = True
    break
if found:
  print("Found some repetition")
else:
  print("No repetitions found")


In [ ]:
# This will take a while

! pip install fasttext
import fasttext.util
print("Installed and loaded the library")
# Download Spanish embeddings using this code or from the link above
fasttext.util.download_model('es', if_exists='ignore')
print("Downloaded embeddings")
# Load them
ft = fasttext.load_model('cc.es.300.bin')
print("Loaded embeddings")

# This is how you get the embedding of a word
vector = ft.get_word_vector('hola')

In [ ]:
! pip install spacy
# Downloading Spanish model
! python -m spacy download es_core_news_sm
import spacy
import es_core_news_sm
nlp = es_core_news_sm.load()
#nlp = spacy.load('es_core_news_sm')
text = "Esto es una frase en español." # "This is a sentence in Spanish."
processed_text = nlp(text)
for t in processed_text:
  print(t)

## 1 Word embedding

The code in this part uses previously implemented functions and the new code was implemented by Frédéric Lauron

In [ ]:

from gensim.corpora.dictionary import Dictionary
from pyemd import emd

# A function that calculates Word Mover's Distance, slightly modified from https://tedboy.github.io/nlps/_modules/gensim/models/word2vec.html#Word2Vec.wmdistance


def wmdistance_es(words1, words2, vectors):
    """
    words1 and words2 are lists of str containing the words of the first and second sentence in a sentence pair.
    These should include only words that you are interested in (e.g., maybe you exclude stopwords or words of some pos)
    vectors is a Magnitude object (created above with the downloaded word embeddings)
    """
    # Remove out-of-vocabulary words.
    if len(words1) == 0 or len(words2) == 0:
        print('At least one of the documents had no words that were'
              'in the vocabulary. Aborting (returning inf).')
        return float('inf')

    dictionary = Dictionary(documents=[words1, words2])
    vocab_len = len(dictionary)

    # Sets for faster look-up.
    docset1 = set(words1)
    docset2 = set(words2)

    # Compute distance matrix.
    distance_matrix = np.zeros((vocab_len, vocab_len))  # , dtype=double)
    for i, t1 in dictionary.items():
        for j, t2 in dictionary.items():
            if not t1 in docset1 or not t2 in docset2:
                continue
            # Compute Euclidean distance between word vectors.
            distance_matrix[i, j] = np.sqrt(
                np.sum((vectors.get_word_vector(t1) - vectors.get_word_vector(t2))**2))

    if np.sum(distance_matrix) == 0.0:
        # `emd` gets stuck if the distance matrix contains only zeros.
        print('The distance matrix is all zeros. Aborting (returning inf).')
        return float('inf')

    def nbow(words):
        d = np.zeros(vocab_len)  # , dtype=double)
        nbow = dictionary.doc2bow(words)  # Word frequencies.
        doc_len = len(words)
        for idx, freq in nbow:
            d[idx] = freq / float(doc_len)  # Normalized word frequencies.
        return d

    # Compute nBOW representation of documents.
    d1 = nbow(words1)
    d2 = nbow(words2)

    # Compute WMD.
    return emd(d1, d2, distance_matrix)


wmdistance_es(["cat","is","blue"],["dog","is","red"],ft)

In [ ]:
# Exercise A.1
stop_words_es = set(stopwords.words('spanish'))

def preprocess_es(dataset):
  preprocessed = {}
  punctuation = [".","?",'"',"'",",","(",")"]
  for key in dataset.keys():
    preprocessed[key] = []
    for i,pair in enumerate(dataset[key]["data"]):
      s1 = pair[0]
      s2 = pair[1]
      s1_preprocessed = [word.lemma_.lower() for word in nlp(s1)]
      s2_preprocessed = [word.lemma_.lower() for word in nlp(s2)]
      s1_preprocessed = [word for word in s1_preprocessed if word not in stop_words_es and word not in punctuation]
      s2_preprocessed = [word for word in s2_preprocessed if word not in stop_words_es and word not in punctuation]
      preprocessed[key].append([s1_preprocessed,s2_preprocessed])




  return preprocessed


def normalize_vector(vec):
    '''Normalize a vector to unit length'''
    norm = np.linalg.norm(vec)
    if norm == 0:
       return vec
    return vec / norm

def assign_distributed_vectors_es(data, vectors, type_features, agreg_func="mean", postag=False, lowercase=False):
  # Write a function like assign_distributional_vectors which uses distributed
  # vectors. You can also simply modify that function.
  '''This functions assigns each sentence a vector and optionally calculates the similarity/distance
  between the representations of s1 and s2.
  Parameters
    data: list of tuples (like dataset['train']['data'])
    vectors: vectors in Magnitude format
    sim_or_dist: bool. If True, we will use a similarity or distance as the only feature. If False,
    we will use the concatenation of the representations of s1 and s2.
    postag: whether we want to apply a postag-based filter to obtain sentence representations
    lowercase: bool. If True, words are lowercased.
    wmdistance: bool. If True, Word Mover's Distance is used.
  Returns:
    features: an array with the data transformed into features '''
  ## TO COMPLETE

  if type_features == "simdist" or type_features == "wmdist":
    features = np.zeros((len(data), 1))
  if type_features == "concatenate":
    features = np.zeros((len(data), vectors.get_dimension()*2))

  for i, (s1, s2) in enumerate(data):
    # Tokenize, lowercase if lowercase=True, and if postag=True, postag s1 and s2
    # TO COMP LETE
    tokens1 = word_tokenize(s1)
    tokens2 = word_tokenize(s2)

    if lowercase:
      tokens1 = [token.lower() for token in tokens1]
      tokens2 = [token.lower() for token in tokens2]

    # Filter out all token that are not "noun", "verb", "adj", "adv"
    if postag:
      tokens1 = [word for word, pos in pos_tag(tokens1, tagset='universal') if pos in [
          'NOUN', 'VERB', 'ADJ', 'ADV']]
      tokens2 = [word for word, pos in pos_tag(tokens2, tagset='universal') if pos in [
          'NOUN', 'VERB', 'ADJ', 'ADV']]

    # Now create two lists, one for each sentence, with the word representations that you want to use
    # You can go through the words (or word, pos) in each sentence and decide whether you keep their representation or not
    # TO COMPLETE
    # unknown word leaved out
    s1vecs = [vectors.get_word_vector(word) for word in tokens1 if word in vectors]
    # unknown word leaved out
    s2vecs = [vectors.get_word_vector(word) for word in tokens2 if word in vectors]

    # It is possible that some sentences will not have any word representation available.
    # We assign them a 0-vector in this case (be careful, because this could result in a cosine of NaN)
    if not s1vecs:
      s1vecs = [np.zeros(vectors.get_dimension())]
    if not s2vecs:
      s2vecs = [np.zeros(vectors.get_dimension())]

    # Aggregate the representations of words in a sentence, for example by averaging them
    # TO COMPLETE
    if agreg_func == "mean":
      s1vec = np.mean(s1vecs, axis=0)
      s2vec = np.mean(s2vecs, axis=0)

    if agreg_func == "std":
      s1vec = np.std(s1vecs, axis=0)
      s2vec = np.std(s2vecs, axis=0)

    if agreg_func == "median":
      s1vec = np.median(s1vecs, axis=0)
      s2vec = np.median(s2vecs, axis=0)

    # Fill in features[i] with the desired feature (one or more similarity/distance measures if sim=True,
    # a concatenation of the representations otherwise)
    # TO COMPLETE
    if type_features == "simdist":
      # Cosine similarity as an example <<<<<
      dot_product = np.dot(s1vec, s2vec)
      norm_s1 = np.linalg.norm(s1vec)
      norm_s2 = np.linalg.norm(s2vec)
      if norm_s1 == 0 or norm_s2 == 0:
        similarity = 0
      else:
        similarity = dot_product / (norm_s1 * norm_s2)
      features[i] = similarity

    if type_features == "concatenate":
      # Concatenate vectors
      s1vec = normalize_vector(s1vec)
      s2vec = normalize_vector(s2vec)
      features[i] = np.concatenate((s1vec, s2vec))

    if type_features == "wmdist":
      features[i] = wmdistance_es(tokens1, tokens2, vectors)

  return features

In [ ]:
#prepr = preprocess_es(mono_data)

from sklearn.linear_model import LinearRegression, RidgeCV, ElasticNetCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
# from sklearn.svm import SVR
# from xgboost import XGBRegressor


def build_model_es(X, Y, type="linear"):
    if (type not in model_types):
        raise Exception("type needs to be a part of these : "+str(model_types))
    if type == "linear":
        model = LinearRegression()
    elif type == "random_forest":
        model = RandomForestRegressor()
    elif type == "ridge":
        model = RidgeCV()
    elif type == "lasso":
        model = LassoCV()
    elif type == "elastic_net":
        model = ElasticNetCV()
    '''
    elif type == "svm":
        model = SVR()
    elif type == "xgboost":
        model = XGBRegressor(objective ='reg:squarederror')
    '''

    model.fit(X, Y)
    return model



for feature in ["simdist", "concatenate", "wmdist"]:
    for agreg_func in ["mean", "median", "std"]:
        for use_pos in [True, False]:
            for use_lower in [True, False]:
                for model_types in ["linear", "ridge", "lasso", "elastic_net"]:

                    features_train = assign_distributed_vectors_es( mono_data['train']['data'], ft, feature, agreg_func, postag=use_pos, lowercase=use_lower)
                    features_dev = assign_distributed_vectors_es(mono_data['test']['data'], ft, feature, agreg_func, postag=use_pos, lowercase=use_lower)

                    Model_caract = f"feature: {feature}, agreg_func: {agreg_func}, use_postag: {use_pos}, use_lower: {use_lower},model_types:{model_types}:"
                    model = build_model_es(features_train, mono_data['train']['scores'], type=model_types)

                    # Evaluation sur le dev set
                    dev_y_pred = model.predict(features_dev)
                    Eval = evaluate(dev_y_pred, mono_data['test']['scores'])

                    f = open("recap.txt", "a")
                    print(Model_caract)
                    f.write(Model_caract+"\n")

                    if model_types == "elastic_net" or model_types == "lasso" or model_types == "ridge":
                        print("alpha:", model.alpha_)
                        f.write("alpha:"+f"{model.alpha_}")
                        f.write("\n")
                    if model_types == "elastic_net":
                        print("ratio:", model.l1_ratio_)
                        f.write("ratio:"+f"{model.l1_ratio_}")
                        f.write("\n")
                    print(Eval)
                    f.write("Eval:"+f"{Eval}")
                    f.write("\n")
                    f.close()

## 2 Word embedding and simple features

This code was written mostly by using functions already implemented in the previous parts, the new parts like the translation were written by Pierre DLVM.

In [ ]:
# Get features
features_train_dist = assign_distributed_vectors_es( mono_data['train']['data'],ft, "simdist", "mean", postag=True, lowercase=True)
features_test_dist= assign_distributed_vectors_es(mono_data['test']['data'], ft, "simdist", "mean", postag=True, lowercase=True)
features_train_wmd = assign_distributed_vectors_es( mono_data['train']['data'],ft, "wmdist", "mean", postag=True, lowercase=True)
features_test_wmd= assign_distributed_vectors_es(mono_data['test']['data'], ft, "wmdist", "mean", postag=True, lowercase=True)

In [ ]:
def extract_features_es(subset_data,ngram_overlap=True,TFIDF=True):
    features = []

    # Compute TF-IDF matrix for all data to be efficient
    tfidf_vectorizer = TfidfVectorizer()
    train_sentences = []

    for pair in subset_data:
        for i in range(2):
            sentence = " ".join(pair[i])
            #print('sentence - pair ', pair, i, sentence)
            train_sentences.append(sentence)
    tfidf_vectorizer.fit(train_sentences)

    for subset in subset_data:
      s1_tokens = subset[0]
      s2_tokens = subset[1]
      s1, s2 = " ".join(s1_tokens), " ".join(s2_tokens)
      current_features = []

      # ngram
      current_features.append(get_ngram_overlap(s1,s2))
      # TFIDF similarity
      current_features.append(tfidf_similarity(s1,s2,tfidf_vectorizer))




      features.append(current_features)

    return np.array(features)

In [ ]:
preprocessed = preprocess_es(mono_data)

In [ ]:
train_features = extract_features_es(preprocessed["train"])
test_features= extract_features_es(preprocessed["test"])

In [ ]:
# Merge features
train_features = np.column_stack((train_features,features_train_dist))
train_features = np.column_stack((train_features,features_train_wmd))

test_features = np.column_stack((test_features,features_test_dist))
test_features = np.column_stack((test_features,features_test_wmd))

In [ ]:
# Pierre DLVM
# Solve issue of values being equal to infinite
for t in train_features:
    for i in range(4):
        if(np.isinf(t[i])):
            t[i] = 0

for t in test_features:
    for i in range(4):
        if(np.isinf(t[i])):
            t[i] = 0

In [ ]:
model = LinearRegression()
model.fit(train_features,train_y)

In [ ]:
evaluate(model.predict(test_features),test_y)

## 3 Simple features on translated dataset

This code was written mostly by using functions already implemented in the previous parts, the new parts like the translation were written by Pierre DLVM.

In [ ]:
! pip install deep-translator

In [ ]:
#from googletrans import Translator, constants
from deep_translator import GoogleTranslator
from pprint import pprint

translator = GoogleTranslator(source='spanish', target='en')
translated_dataset = {}

translated_dataset["train"] = {}
translated_dataset["train"]["data"] = []
all_sentences =[]


for pair in mono_data["train"]["data"]:
    all_sentences.extend(pair)
# Batch translate all sentences
translations = translator.translate_batch(all_sentences,src="spanish",dest="english")
np.save("translations_train.npy",translations)

for pair in mono_data["test"]["data"]:
    all_sentences.extend(pair)
# Batch translate all sentences
translations = translator.translate_batch(all_sentences,src="spanish",dest="english")
np.save("translations_test.npy",translations)


In [ ]:
#translated_dataset["train"]["data"]
#np.save("translations_test.npy",translations)
train= np.load("translations_train.npy")
test= np.load("translations_test.npy")
translated_dataset = {}
translated_dataset["train"] = {}
translated_dataset["test"] = {}
translated_dataset["train"]["data"] = []
translated_dataset["test"]["data"] = []

for i in range(int(len(train)/2)):
    translated_dataset["train"]["data"].append([train[2*i],train[2*i+1]])

for i in range(int(len(test)/2)):
    translated_dataset["test"]["data"].append([test[2*i],test[2*i+1]])

In [ ]:
default_preprocessed , wordnet_preprocessed = preprocess_dataset(dataset=translated_dataset,
                                                               remove_punctuation=True,
                                                               use_stemming=False,
                                                               use_lemmatization=True,
                                                               remove_stopwords=True,
                                                               remASCII=False)

In [ ]:
np.save("translated_dataset_preprocessed_default.npy",default_preprocessed)
np.save("translated_dataset_preprocessed_wordnet.npy",wordnet_preprocessed)

In [ ]:
# Load if using loaded data

#default_preprocessed = np.load("translated_dataset_preprocessed_default.npy",allow_pickle=True).item()
#wordnet_preprocessed = np.load("translated_dataset_preprocessed_wordnet.npy",allow_pickle=True).item()

In [ ]:
train_features = extract_features(default_preprocessed["train"]["data"],wordnet_preprocessed["train"]["data"])
test_features = extract_features(default_preprocessed["test"]["data"],wordnet_preprocessed["test"]["data"])

In [ ]:
#np.save("features_train_es.npy",train_features)
#np.save("features_test_es.npy",train_features)

In [ ]:
#train_features = np.load("features_train_es.npy")
#test_features = np.load("features_test_es.npy")

In [ ]:
param_grid_svm = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf']}

base_model = SVR()
# times of compute important with GridSearchCV(). We'll use RandomizedSearch() to control the number of iteration
# grid_search = RandomizedSearchCV(model, param_grid_svm, n_iter=5, cv=5, scoring='r2', random_state=42)
grid_search = GridSearchCV(base_model, param_grid_svm, cv=5, scoring='r2')
grid_search.fit(train_features, train_y)
model = grid_search.best_estimator_


In [ ]:
evaluate(model.predict(test_features),test_y)